<a href="https://colab.research.google.com/github/praveentn/hgwxx7/blob/master/projects/ImageSegmentation_fcn_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CellStrat AI Lab

##Semantic Image Segmentation - FCN model for Kitty Roads dataset

In [0]:
# %load main.py
#--------------------------
# USER-SPECIFIED DATA
#CellStrat
#Reference blog : https://medium.com/nanonets/how-to-do-image-segmentation-using-deep-learning-c673cc5862ef
# Dataset download : Download the Kitti Road dataset from http://www.cvlibs.net/download.php?file=data_road.zip
# In this model we will run 40 epochs on the entire dataset. In Colab it takes 10-15 minutes to run
# On a personal computer, just with 5 training images and 40 training epochs, this program took 6-8 hours.
# Better to run it on the Cloud
#--------------------------

# Tune these parameters

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/ImageSegmentation/FCN')
!pwd

/content/gdrive/My Drive/Colab Notebooks/ImageSegmentation/FCN


In [4]:
pip install Pillow

In [5]:
!pip install scikit-image
#pip install skimage
#CellStrat - use older version of scipy which has APIs like imresize, imsave, toimage, imsave etc.
!pip install scipy==1.2.1

     |████████████████████████████████| 24.8MB 25.2MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.0
    Uninstalling scipy-1.3.0:
      Successfully uninstalled scipy-1.3.0


In [0]:
import tensorflow as tf
import helper

tf.reset_default_graph()

In [7]:
tf.reset_default_graph()

#CellStrat
num_classes = 2
NUMBER_OF_CLASSES = 2
image_shape = (160, 576)
IMAGE_SHAPE = (160, 576)
EPOCHS = 40
BATCH_SIZE = 16
DROPOUT = 0.75

# Specify these directory paths

data_dir = './data'
runs_dir = './runs'
training_dir ='./data/data_road/training'
vgg_path = './data/vgg'

#--------------------------
# PLACEHOLDER TENSORS
#--------------------------

correct_label = tf.placeholder(tf.float32, [None, IMAGE_SHAPE[0], IMAGE_SHAPE[1], NUMBER_OF_CLASSES])
learning_rate = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32)

#--------------------------
# FUNCTIONS
#--------------------------

def load_vgg(sess, vgg_path):
  
  # load the model and weights
  model = tf.saved_model.loader.load(sess, ['vgg16'], vgg_path)

  # Get Tensors to be returned from graph
  graph = tf.get_default_graph()
  image_input = graph.get_tensor_by_name('image_input:0')
  keep_prob = graph.get_tensor_by_name('keep_prob:0')
  layer3 = graph.get_tensor_by_name('layer3_out:0')
  layer4 = graph.get_tensor_by_name('layer4_out:0')
  layer7 = graph.get_tensor_by_name('layer7_out:0')

  return image_input, keep_prob, layer3, layer4, layer7

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
   
    # Use a shorter variable name for simplicity
    layer3, layer4, layer7 = vgg_layer3_out, vgg_layer4_out, vgg_layer7_out

    # Apply 1x1 convolution in place of fully connected layer
    fcn8 = tf.layers.conv2d(layer7, filters=num_classes, kernel_size=1, name="fcn8")

    # Upsample fcn8 with size depth=(4096?) to match size of layer 4 so that we can add skip connection with 4th layer
    fcn9 = tf.layers.conv2d_transpose(fcn8, filters=layer4.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn9")

    # Add a skip connection between current final layer fcn8 and 4th layer
    fcn9_skip_connected = tf.add(fcn9, layer4, name="fcn9_plus_vgg_layer4")

    # Upsample again
    fcn10 = tf.layers.conv2d_transpose(fcn9_skip_connected, filters=layer3.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn10_conv2d")

    # Add skip connection
    fcn10_skip_connected = tf.add(fcn10, layer3, name="fcn10_plus_vgg_layer3")

    # Upsample again
    fcn11 = tf.layers.conv2d_transpose(fcn10_skip_connected, filters=num_classes,
    kernel_size=16, strides=(8, 8), padding='SAME', name="fcn11")

    return fcn11

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
  
  # Reshape 4D tensors to 2D, each row represents a pixel, each column a class
  logits = tf.reshape(nn_last_layer, (-1, num_classes), name="fcn_logits")
  correct_label_reshaped = tf.reshape(correct_label, (-1, num_classes))

  # Calculate distance from actual labels using cross entropy
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label_reshaped[:])
  # Take mean for total loss
  loss_op = tf.reduce_mean(cross_entropy, name="fcn_loss")

  # The model implements this operation to find the weights/parameters that would yield correct pixel labels
  train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op, name="fcn_train_op")

  return logits, train_op, loss_op

def train_nn(sess, epochs, batch_size, get_batches_fn, train_op,
             cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):

  keep_prob_value = 0.5
  learning_rate_value = 0.001
  for epoch in range(epochs):
      # Create function to get batches
      total_loss = 0
      for X_batch, gt_batch in get_batches_fn(batch_size):

          loss, _ = sess.run([cross_entropy_loss, train_op],
          feed_dict={input_image: X_batch, correct_label: gt_batch,
          keep_prob: keep_prob_value, learning_rate:learning_rate_value})

          total_loss += loss;

      print("EPOCH {} ...".format(epoch + 1))
      print("Loss = {:.3f}".format(total_loss))
      print()

def run():
  
  # Download pretrained vgg model
  helper.maybe_download_pretrained_vgg(data_dir)

  # A function to get batches
  get_batches_fn = helper.gen_batch_function(training_dir, image_shape)
  
  with tf.Session() as session:
        
    # Returns the three layers, keep probability and input layer from the vgg architecture
    image_input, keep_prob, layer3, layer4, layer7 = load_vgg(session, vgg_path)

    # The resulting network architecture from adding a decoder on top of the given vgg model
    model_output = layers(layer3, layer4, layer7, num_classes)

    # Returns the output logits, training operation and cost operation to be used
    # - logits: each row represents a pixel, each column a class
    # - train_op: function used to get the right parameters to the model to correctly label the pixels
    # - cross_entropy_loss: function outputting the cost which we are minimizing, lower cost should yield higher accuracy
    logits, train_op, cross_entropy_loss = optimize(model_output, correct_label, learning_rate, num_classes)
    
    # Initialize all variables
    session.run(tf.global_variables_initializer())
    session.run(tf.local_variables_initializer())

    print("Model build successful, starting training")

    # Train the neural network
    train_nn(session, EPOCHS, BATCH_SIZE, get_batches_fn, 
             train_op, cross_entropy_loss, image_input,
             correct_label, keep_prob, learning_rate)

    # Run the model with the test images and save each painted output image (roads painted green)
    helper.save_inference_samples(runs_dir, data_dir, session, image_shape, logits, keep_prob, image_input)
    
    print("All done!")

#--------------------------
# MAIN
#--------------------------
if __name__ == '__main__':
    run()

W0810 10:45:39.993680 139732949751680 deprecation.py:323] From <ipython-input-7-cd36cc3aefdf>:34: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0810 10:45:50.004604 139732949751680 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0810 10:45:59.081947 139732949751680 deprecation.py:323] From <ipython-input-7-cd36cc3aefdf>:52: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed

Model build successful, starting training
EPOCH 1 ...
Loss = 0.000

EPOCH 2 ...
Loss = 0.000

EPOCH 3 ...
Loss = 0.000

EPOCH 4 ...
Loss = 0.000

EPOCH 5 ...
Loss = 0.000

EPOCH 6 ...
Loss = 0.000

EPOCH 7 ...
Loss = 0.000

EPOCH 8 ...
Loss = 0.000

EPOCH 9 ...
Loss = 0.000

EPOCH 10 ...
Loss = 0.000

EPOCH 11 ...
Loss = 0.000

EPOCH 12 ...
Loss = 0.000

EPOCH 13 ...
Loss = 0.000

EPOCH 14 ...
Loss = 0.000

EPOCH 15 ...
Loss = 0.000

EPOCH 16 ...
Loss = 0.000

EPOCH 17 ...
Loss = 0.000

EPOCH 18 ...
Loss = 0.000

EPOCH 19 ...
Loss = 0.000

EPOCH 20 ...
Loss = 0.000

EPOCH 21 ...
Loss = 0.000

EPOCH 22 ...
Loss = 0.000

EPOCH 23 ...
Loss = 0.000

EPOCH 24 ...
Loss = 0.000

EPOCH 25 ...
Loss = 0.000

EPOCH 26 ...
Loss = 0.000

EPOCH 27 ...
Loss = 0.000

EPOCH 28 ...
Loss = 0.000

EPOCH 29 ...
Loss = 0.000

EPOCH 30 ...
Loss = 0.000

EPOCH 31 ...
Loss = 0.000

EPOCH 32 ...
Loss = 0.000

EPOCH 33 ...
Loss = 0.000

EPOCH 34 ...
Loss = 0.000

EPOCH 35 ...
Loss = 0.000

EPOCH 36 ...
Loss = 0.